**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# Start

Deployment and Service

https://kubernetes.io/docs/concepts/services-networking/connect-applications-service/

In [1]:
kubectl get pods -A

NAMESPACE     NAME                               READY   STATUS    RESTARTS   AGE
kube-system   coredns-74ff55c5b-hq2hm            1/1     Running   1          18h
kube-system   etcd-minikube                      1/1     Running   1          18h
kube-system   kube-apiserver-minikube            1/1     Running   1          18h
kube-system   kube-controller-manager-minikube   1/1     Running   1          18h
kube-system   kube-proxy-lv5cn                   1/1     Running   1          18h
kube-system   kube-scheduler-minikube            1/1     Running   1          18h
kube-system   storage-provisioner                1/1     Running   3          18h


# Warning

**we install software from an unknown source**

In [2]:
kubectl create -f https://raw.githubusercontent.com/kubernetes/website/master/content/en/examples/service/networking/run-my-nginx.yaml

deployment.apps/my-nginx created


In [3]:
kubectl create -f https://raw.githubusercontent.com/kubernetes/website/master/content/en/examples/service/networking/nginx-svc.yaml

service/my-nginx created


In [6]:
kubectl get all

NAME                            READY   STATUS    RESTARTS   AGE
pod/my-nginx-5b56ccd65f-9vrdg   1/1     Running   0          5m6s
pod/my-nginx-5b56ccd65f-zl5ss   1/1     Running   0          5m6s

NAME                 TYPE           CLUSTER-IP     EXTERNAL-IP    PORT(S)        AGE
service/kubernetes   ClusterIP      10.96.0.1      <none>         443/TCP        18h
service/my-nginx     LoadBalancer   10.109.23.43   10.109.23.43   80:31563/TCP   5m3s

NAME                       READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/my-nginx   2/2     2            2           5m6s

NAME                                  DESIRED   CURRENT   READY   AGE
replicaset.apps/my-nginx-5b56ccd65f   2         2         2       5m6s


In [7]:
curl https://raw.githubusercontent.com/kubernetes/website/master/content/en/examples/service/networking/nginx-svc.yaml

apiVersion: v1
kind: Service
metadata:
  name: my-nginx
  labels:
    run: my-nginx
spec:
  ports:
  - port: 80
    protocol: TCP
  selector:
    run: my-nginx


In [8]:
curl https://raw.githubusercontent.com/kubernetes/website/master/content/en/examples/service/networking/run-my-nginx.yaml

apiVersion: apps/v1
kind: Deployment
metadata:
  name: my-nginx
spec:
  selector:
    matchLabels:
      run: my-nginx
  replicas: 2
  template:
    metadata:
      labels:
        run: my-nginx
    spec:
      containers:
      - name: my-nginx
        image: nginx
        ports:
        - containerPort: 80



In [9]:
kubectl get service my-nginx

NAME       TYPE           CLUSTER-IP     EXTERNAL-IP    PORT(S)        AGE
my-nginx   LoadBalancer   10.109.23.43   10.109.23.43   80:31563/TCP   8m39s


In [10]:
kubectl get service my-nginx -o yaml

apiVersion: v1
kind: Service
metadata:
  creationTimestamp: "2021-04-28T09:12:56Z"
  labels:
    run: my-nginx
  name: my-nginx
  namespace: default
  resourceVersion: "18357"
  uid: 59dfd706-e080-44f7-af23-633301b6a1d3
spec:
  clusterIP: 10.109.23.43
  clusterIPs:
  - 10.109.23.43
  externalTrafficPolicy: Cluster
  ports:
  - nodePort: 31563
    port: 80
    protocol: TCP
    targetPort: 80
  selector:
    run: my-nginx
  sessionAffinity: None
  type: LoadBalancer
status:
  loadBalancer:
    ingress:
    - ip: 10.109.23.43


In [11]:
kubectl get service my-nginx -o jsonpath='{.spec.clusterIP}'

10.109.23.43

In [12]:
IP=$(kubectl get service my-nginx -o jsonpath='{.spec.clusterIP}')
curl $IP

<!DOCTYPE html>
<html>
<head>
<title>Welcome to nginx!</title>
<style>
    body {
        width: 35em;
        margin: 0 auto;
        font-family: Tahoma, Verdana, Arial, sans-serif;
    }
</style>
</head>
<body>
<h1>Welcome to nginx!</h1>
<p>If you see this page, the nginx web server is successfully installed and
working. Further configuration is required.</p>

<p>For online documentation and support please refer to
<a href="http://nginx.org/">nginx.org</a>.<br/>
Commercial support is available at
<a href="http://nginx.com/">nginx.com</a>.</p>

<p><em>Thank you for using nginx.</em></p>
</body>
</html>


In [13]:
echo http://$IP

http://10.109.23.43


In [15]:
kubectl get service -A -o jsonpath='{.items[*].spec.clusterIP}'

10.96.0.1 10.109.23.43 10.96.0.10

In [1]:
kubectl get pods --all-namespaces -o go-template \
--template="{{range .items}}{{.metadata.namespace}}/{{.metadata.name}}:{{println}}{{range .spec.containers}}  {{.image}}:{{.securityContext}}
{{end}}{{end}}" | grep --color=always -E '(^|privileged)'


default/my-nginx-5b56ccd65f-9vrdg:
  nginx:<no value>
default/my-nginx-5b56ccd65f-zl5ss:
  nginx:<no value>
kube-system/coredns-74ff55c5b-hq2hm:
  k8s.gcr.io/coredns:1.7.0:map[allowPrivilegeEscalation:false capabilities:map[add:[NET_BIND_SERVICE] drop:[all]] readOnlyRootFilesystem:true]
kube-system/etcd-minikube:
  k8s.gcr.io/etcd:3.4.13-0:<no value>
kube-system/kube-apiserver-minikube:
  k8s.gcr.io/kube-apiserver:v1.20.2:<no value>
kube-system/kube-controller-manager-minikube:
  k8s.gcr.io/kube-controller-manager:v1.20.2:<no value>
kube-system/kube-proxy-lv5cn:
  k8s.gcr.io/kube-proxy:v1.20.2:map[privileged:true]
kube-system/kube-scheduler-minikube:
  k8s.gcr.io/kube-scheduler:v1.20.2:<no value>
kube-system/storage-provisioner:
  gcr.io/k8s-minikube/storage-provisioner:v5:<no value>
kubernetes-dashboard/dashboard-metrics-scraper-f6647bd8c-cst2g:
  kubernetesui/metrics-scraper:v1.0.4@sha256:555981a24f184420f3be0c79d4efb6c948a85cfce84034f85a563f4151a81cbf:map[allowPrivilegeEscalation:fa